In [ ]:
# Imports: first run pip install below
import csv
import pandas as pd
from pyspark.sql import SparkSession


In [ ]:
#!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 70.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=c4f29dd0a2aa50c7b8c2b606cc5c404d7e55082fac9e145b3c95fe6c8844398f
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
bitcoin_path = "Resources/coin_Bitcoin.csv"
# tweets_path = "Resources/Bitcoin_tweets.csv"

# Read World Happiness Data Using File Path
#bitcoin_df = pd.read_csv(bitcoin_path)
#tweets_data = pd.read_csv(tweets_path)

In [ ]:
bitcoin_df.head()

NameError: ignored

In [ ]:
# tweets_df.head(10)

In [ ]:
bitcoin_df = bitcoin_df.drop(columns = ["SNo","Name","Symbol"])

NameError: ignored

In [ ]:
bitcoin_df.head()

NameError: ignored

In [ ]:
#reading coin_Bitcoin into pyspark
spark = SparkSession.builder.appName("read_csv").getOrCreate()
bitcoin_data = spark.read.csv("gdrive/MyDrive/coin_Bitcoin.csv",sep = ",",inferSchema = True, header = True)

In [ ]:
# converting dataframe from pyspark to Pandas
bitcoin_df = bitcoin_data.toPandas()

In [ ]:
#Display DF as Pandas, line below removes 3 columns
bitcoin_df.head()

SNo     Name Symbol                 Date        High         Low  \
0    1  Bitcoin    BTC  2013-04-29 23:59:59  147.488007  134.000000   
1    2  Bitcoin    BTC  2013-04-30 23:59:59  146.929993  134.050003   
2    3  Bitcoin    BTC  2013-05-01 23:59:59  139.889999  107.720001   
3    4  Bitcoin    BTC  2013-05-02 23:59:59  125.599998   92.281898   
4    5  Bitcoin    BTC  2013-05-03 23:59:59  108.127998   79.099998   

         Open       Close  Volume     Marketcap  
0  134.444000  144.539993     0.0  1.603769e+09  
1  144.000000  139.000000     0.0  1.542813e+09  
2  139.000000  116.989998     0.0  1.298955e+09  
3  116.379997  105.209999     0.0  1.168517e+09  
4  106.250000   97.750000     0.0  1.085995e+09

In [ ]:
#Removing 3 columns
bitcoin_df = bitcoin_df.drop(columns = ["SNo","Name","Symbol"])

In [ ]:
# reading Bitcoin_tweets into pyspark, attemping to remove double lines
tweets_data = spark.read.csv("gdrive/MyDrive/Bitcoin_tweets.csv",sep = ",",inferSchema = True, header = True,multiLine= True, escape = '\"')

In [ ]:
# display DF without double lines
tweets_data.show(5,truncate=False)

+--------------------------------------------+---------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------+--------------+------------+---------------+-------------+-------------------+----------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------+-------------------+----------+
|user_name                                   |user_location  |user_description                                                                                                                                                |user_created       |user_followers|user_friends|user_favourites|user_verified|date               |text                                                                                                                                         

In [ ]:
#dropping coulmns from DF
tweets_data = tweets_data.drop("user_name", "hashtags", "source", "is_retweet", "user_created","user_location" )

In [ ]:
#checking data types to convert
tweets_data.printSchema()

root
 |-- user_description: string (nullable = true)
 |-- user_followers: string (nullable = true)
 |-- user_friends: string (nullable = true)
 |-- user_favourites: string (nullable = true)
 |-- user_verified: string (nullable = true)
 |-- date: string (nullable = true)
 |-- text: string (nullable = true)



In [ ]:
#displaying DF with dropped columns
tweets_data.show(5,truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------+------------+---------------+-------------+-------------------+----------------------------------------------------------------------------------------------------------------------------------------------+
|user_description                                                                                                                                                |user_followers|user_friends|user_favourites|user_verified|date               |text                                                                                                                                          |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------+------------+---------------+-------------+------------

In [ ]:
#removing rows with missing values: NA/NULL .show(truncate=False)
tweets_data = tweets_data.na.drop(how="any")

In [ ]:
#display DF without NULL
tweets_data.show(10,truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------+------------+---------------+-------------+-------------------+-------------------------------------------------------------------------------------------------------------------------------------------------+
|user_description                                                                                                                                                |user_followers|user_friends|user_favourites|user_verified|date               |text                                                                                                                                             |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------+------------+---------------+-------------+------

In [ ]:
# displaying the current data type to change
tweets_data.printSchema()

root
 |-- user_description: string (nullable = true)
 |-- user_followers: string (nullable = true)
 |-- user_friends: string (nullable = true)
 |-- user_favourites: string (nullable = true)
 |-- user_verified: string (nullable = true)
 |-- date: string (nullable = true)
 |-- text: string (nullable = true)



In [ ]:
#changing the following to appropriate data types and diplaying correctly
from pyspark.sql.functions import col
from pyspark.sql.types import StringType,BooleanType,DateType, IntegerType
tweets_data = tweets_data.withColumn("user_followers",col("user_followers").cast(IntegerType())) \
    .withColumn("user_friends",col("user_friends").cast(IntegerType())) \
    .withColumn("user_favourites",col("user_favourites").cast(IntegerType())) \
    .withColumn("date",col("date").cast(DateType()))
tweets_data.printSchema()

root
 |-- user_description: string (nullable = true)
 |-- user_followers: integer (nullable = true)
 |-- user_friends: integer (nullable = true)
 |-- user_favourites: integer (nullable = true)
 |-- user_verified: string (nullable = true)
 |-- date: date (nullable = true)
 |-- text: string (nullable = true)



In [ ]:
#changing names of cloumns
tweets_data = tweets_data.withColumnRenamed("user_description","Bio") \
    .withColumnRenamed("user_followers","Followers") \
    .withColumnRenamed("user_friends","Following") \
    .withColumnRenamed("user_favourites","Likes") \
    .withColumnRenamed("user_verified","Verified User") \
    .withColumnRenamed("date","Date") \
    .withColumnRenamed("text","Tweet") 
tweets_data.printSchema()

root
 |-- Bio: string (nullable = true)
 |-- Followers: integer (nullable = true)
 |-- Following: integer (nullable = true)
 |-- Likes: integer (nullable = true)
 |-- Verified User: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- Tweet: string (nullable = true)



In [ ]:
#displaying new column names
tweets_data.show(5,truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------+---------+---------+-----+-------------+----------+----------------------------------------------------------------------------------------------------------------------------------------------+
|Bio                                                                                                                                                             |Followers|Following|Likes|Verified User|Date      |Tweet                                                                                                                                         |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------+---------+---------+-----+-------------+----------+-------------------------------------------------------------------------

In [ ]:
from pyspark.sql.functions import desc
tweets_data.groupBy("Date").sum("Likes").show()
#tweets_data.groupBy("Date").count().show(10)
#tweets_data.groupBy("Date").max("Likes").show()
#tweets_data.count().filter("Likes").sort(desc("Likes")).show

+----------+----------+
|      Date|sum(Likes)|
+----------+----------+
|2021-06-22| 352176410|
|2021-11-25| 200579166|
|2021-07-20| 255517385|
|2021-07-30| 244843586|
|2022-05-26| 109759666|
|2021-07-17| 119150467|
|2022-03-16| 246814936|
|2021-04-24|  18160630|
|2021-07-23| 462135352|
|2021-02-15|  29916623|
|2022-03-15| 149757850|
|2021-08-21|  81507219|
|2021-07-19| 231502452|
|2022-02-16|  16354844|
|2022-05-01| 330266650|
|2021-11-11| 120246331|
|2022-04-27|  12070735|
|2022-03-22|    692848|
|2021-05-29|  16838480|
|2022-05-23|  34009313|
+----------+----------+
only showing top 20 rows



In [ ]:
tweets_data.groupBy("Date").count().show(10)

+----------+-----+
|      Date|count|
+----------+-----+
|2021-06-22|41229|
|2021-11-25|32446|
|2021-07-20|28271|
|2021-07-30|34064|
|2022-05-26|18870|
|2021-07-17|13756|
|2022-03-16|36135|
|2021-04-24| 3703|
|2021-07-23|39262|
|2021-02-15| 4323|
+----------+-----+
only showing top 10 rows



In [ ]:
tweets_data.groupBy("Date").max("Likes").show()

+----------+----------+
|      Date|max(Likes)|
+----------+----------+
|2021-06-22|   1178851|
|2021-11-25|    468277|
|2021-07-20|   1091965|
|2021-07-30|    458537|
|2022-05-26|    350505|
|2021-07-17|    525350|
|2022-03-16|    479351|
|2021-04-24|    297752|
|2021-07-23|   1119654|
|2021-02-15|    338292|
|2022-03-15|    399286|
|2021-08-21|    437289|
|2021-07-19|    525350|
|2022-02-16|    402634|
|2022-05-01|   1185823|
|2021-11-11|    588543|
|2022-04-27|    218936|
|2022-03-22|    239934|
|2021-05-29|    287483|
|2022-05-23|    421395|
+----------+----------+
only showing top 20 rows

